## export library results

This scritp is used to group the verification results by library and generate the Result JSON summary file. 

In [1]:
%env AIIDA_PATH=/home/jyu/micromamba/envs/aiida-sssp-dev/etc/.aiida

%load_ext aiida
%aiida

env: AIIDA_PATH=/home/jyu/micromamba/envs/aiida-sssp-dev/etc/.aiida


Loaded AiiDA DB environment - profile name: prod-sssp-compare-04-08.

In [2]:
from aiida import load_profile

load_profile("prod-sssp-compare-04-08")

Profile<uuid='14f03741f5cf47d0914ad118cf51a48a' name='prod-sssp-compare-04-08'>

In [3]:
import json
with open("../sssp_paper_plots/library-data/results-oxides-verification-PBE-v1-quantum_espresso-SSSP-1.3-PBE-precision.json", "r") as f:
    acwf_oxides_results = json.load(f)
    
with open("../sssp_paper_plots/library-data/results-unaries-verification-PBE-v1-quantum_espresso-SSSP-1.3-PBE-precision.json", "r") as f:
    acwf_unaries_results = json.load(f)

In [4]:
# create group 
from aiida import orm
try:
    group = orm.Group(label='sssp-prec-1.3.0-PBE').store()
except:
    # group already exists
    group = orm.load_group(label='sssp-prec-1.3.0-PBE')

In [5]:
import json

# Add notes to group by element-library mapping
# the key is the elenment symbol, the value is the library name contained in the label name: `sg15` or `oncvpsp4.dojo` depending on the details of how the name is 
# distinctively constructed.
# Support the wildcard `%` and `_` for directly used in AiiDA QueryBuilder
SSSP_PREC_1_3_0_PBE_ELEMENT_LIBRARY_MAPPING ={
}

MISSING_ELEMENTS = ['At', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Cd'] # remove Cd for now
LANTHANIUM_ELEMENTS = ['La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu']
MISSING_ELEMENTS.extend(LANTHANIUM_ELEMENTS)

with open('../libraries-pbe/MIX-SSSP-precision-1.3.0-recollected/SSSP_1.3.0_PBE_precision.json') as f:
    data = json.load(f)
    
def pseudo2name(pseudo):
    if pseudo.startswith('SG15'):
        return 'oncvpsp3.sg15'
    elif pseudo.startswith('GBRV'):
        return 'uspp.gbrv'
    elif pseudo == '100PAW':
        return 'paw.%.psl.v1.0.0-high'
    elif pseudo == '100US':
        return 'us.%.psl.v1.0.0-high'
    elif pseudo == '031US':
        return 'us.%.psl.v0.%'
    elif pseudo == '031PAW':
        return 'paw.%.psl.v0.%'
    elif pseudo == 'Dojo':
        return 'oncvpsp3.dojo'
    elif pseudo == '100PAW-low':
        return 'paw.%.psl.v1.0.0-low'
    elif pseudo == 'US100-low': # typo in the original file
        return 'us.%.psl.v1.0.0-low'
    
    
for element, d in data.items():
    if element in MISSING_ELEMENTS:
        continue
    lib_str = pseudo2name(d['pseudopotential'])
    if lib_str is None:
        print(f"Warning: {element} is not supported yet")
        continue
    SSSP_PREC_1_3_0_PBE_ELEMENT_LIBRARY_MAPPING[element] = lib_str
    
    
# use the Dojo 0.5 for Ba
SSSP_PREC_1_3_0_PBE_ELEMENT_LIBRARY_MAPPING['Ba'] = 'oncvpsp4.dojo.%v0.5.0'

# I is regenerated using oncvpsp4.sg15
SSSP_PREC_1_3_0_PBE_ELEMENT_LIBRARY_MAPPING['I'] = 'oncvpsp4.sg15'

In [6]:
SSSP_PREC_1_3_0_PBE_ELEMENT_LIBRARY_MAPPING

{'Ag': 'oncvpsp3.sg15',
 'Al': 'paw.%.psl.v1.0.0-high',
 'Ar': 'paw.%.psl.v1.0.0-high',
 'As': 'oncvpsp3.dojo',
 'Au': 'oncvpsp3.sg15',
 'B': 'uspp.gbrv',
 'Ba': 'oncvpsp4.dojo.%v0.5.0',
 'Be': 'oncvpsp3.sg15',
 'Bi': 'uspp.gbrv',
 'Br': 'uspp.gbrv',
 'C': 'paw.%.psl.v1.0.0-high',
 'Ca': 'uspp.gbrv',
 'Cl': 'us.%.psl.v1.0.0-high',
 'Co': 'uspp.gbrv',
 'Cr': 'uspp.gbrv',
 'Cs': 'oncvpsp3.dojo',
 'Cu': 'paw.%.psl.v1.0.0-low',
 'F': 'oncvpsp3.dojo',
 'Fe': 'paw.%.psl.v0.%',
 'Ga': 'paw.%.psl.v1.0.0-high',
 'Ge': 'uspp.gbrv',
 'H': 'oncvpsp3.sg15',
 'He': 'oncvpsp3.sg15',
 'Hf': 'oncvpsp3.dojo',
 'Hg': 'uspp.gbrv',
 'I': 'oncvpsp4.sg15',
 'In': 'us.%.psl.v0.%',
 'Ir': 'us.%.psl.v1.0.0-high',
 'K': 'paw.%.psl.v1.0.0-high',
 'Kr': 'paw.%.psl.v1.0.0-high',
 'Li': 'uspp.gbrv',
 'Mg': 'uspp.gbrv',
 'Mn': 'uspp.gbrv',
 'Mo': 'oncvpsp3.sg15',
 'N': 'oncvpsp3.dojo',
 'Na': 'paw.%.psl.v1.0.0-low',
 'Nb': 'paw.%.psl.v0.%',
 'Ne': 'paw.%.psl.v1.0.0-high',
 'Ni': 'uspp.gbrv',
 'O': 'paw.%.psl.v0.%',
 

In [7]:
def find_node_by_element_lib(element, lib_str):
    qb = orm.QueryBuilder()
    qb.append(orm.WorkChainNode, filters={'extras.label': {'ilike': f'% {element}.%{lib_str}%'}})
    nodes = qb.all(flat=True)

    if len(nodes) == 0:
        raise Exception(f"No nodes found for {element} with {lib_str}. Check the mapping and the query.")
    elif len(nodes) > 1:
        if element == 'Cd':
            # there are two, redandant, nodes for Cd
            node = nodes[0]
            return node
            
        raise Exception(f"More than one node found for {element} with {lib_str}. Check the mapping and the query.")
    else:
        node = nodes[0]
        
    return node

In [8]:
for key, value in SSSP_PREC_1_3_0_PBE_ELEMENT_LIBRARY_MAPPING.items():
    node = find_node_by_element_lib(key, value)
    group.add_nodes(node)

In [ ]:
# For test
# element = 'Cd'
# lib_str = 'oncvpsp3.sg15'
# qb = orm.QueryBuilder()
# qb.append(orm.WorkChainNode, filters={'extras.label': {'ilike': f'% {element}.%{lib_str}%'}})
# nodes = qb.all(flat=True)

In [9]:
len(list(group.nodes))

69

In [10]:
all_unaries_results = {
    "BM_fit_data": {},
    "num_atoms_in_sim_cell": {},
    'script_version': '0.0.4',
}

for n in group.nodes:
    delta_res = n.outputs.accuracy.delta
    element = n.base.extras.get('element')

    # unaries
    for conf in ["BCC", "FCC", "Diamond", "SC"]:
        try:
            res = delta_res[conf].output_parameters.get_dict()
            V0, B0, B1 = res["birch_murnaghan_results"]
            
            all_unaries_results["BM_fit_data"][f"{element}-X/{conf}"] = {
                "E0": 0,
                "bulk_deriv": B1,
                "bulk_modulus_ev_ang3": B0,
                "min_volume": V0,
                "residuals": 0
            }
            all_unaries_results["num_atoms_in_sim_cell"][f"{element}-X/{conf}"] = res["natoms"]
        except:
            print(f"Warning: {element} {conf} not found, get from acwf results for the moment")
            all_unaries_results["BM_fit_data"][f"{element}-X/{conf}"] = acwf_unaries_results["BM_fit_data"][f"{element}-X/{conf}"]
            all_unaries_results["num_atoms_in_sim_cell"][f"{element}-X/{conf}"] = acwf_unaries_results["num_atoms_in_sim_cell"][f"{element}-X/{conf}"]

In [11]:
# Missing elements
for element in MISSING_ELEMENTS:
    for conf in ["BCC", "FCC", "Diamond", "SC"]:
        all_unaries_results["BM_fit_data"][f"{element}-X/{conf}"] = acwf_unaries_results["BM_fit_data"][f"{element}-X/{conf}"]
        all_unaries_results["num_atoms_in_sim_cell"][f"{element}-X/{conf}"] = acwf_unaries_results["num_atoms_in_sim_cell"][f"{element}-X/{conf}"]

In [12]:
with open("results-unaries-verification-PBE-v1-quantum_espresso-SSSP-1.3.0-PBE-precision-tmp.json", "w") as f:
    json.dump(all_unaries_results, f, indent=4)

In [13]:
all_oxides_results = {
    "BM_fit_data": {},
    "num_atoms_in_sim_cell": {},
    'script_version': '0.0.4',
}

for n in group.nodes:
    delta_res = n.outputs.accuracy.delta
    element = n.base.extras.get('element')

    # unaries
    for conf in ["XO", "XO2", "XO3", "X2O", "X2O3", "X2O5"]:
        try:
            res = delta_res[conf].output_parameters.get_dict()
            V0, B0, B1 = res["birch_murnaghan_results"]
            
            all_oxides_results["BM_fit_data"][f"{element}-{conf}"] = {
                "E0": 0,
                "bulk_deriv": B1,
                "bulk_modulus_ev_ang3": B0,
                "min_volume": V0,
                "residuals": 0
            }
            all_oxides_results["num_atoms_in_sim_cell"][f"{element}-{conf}"] = res["natoms"]
        except:
            print(f"Warning: {element} {conf} not found, get from acwf results for the moment")
            all_oxides_results["BM_fit_data"][f"{element}-{conf}"] = acwf_oxides_results["BM_fit_data"][f"{element}-{conf}"]
            all_oxides_results["num_atoms_in_sim_cell"][f"{element}-{conf}"] = acwf_oxides_results["num_atoms_in_sim_cell"][f"{element}-{conf}"]

In [14]:
# Missing elements
for element in MISSING_ELEMENTS:
    for conf in ["XO", "XO2", "XO3", "X2O", "X2O3", "X2O5"]:
        all_oxides_results["BM_fit_data"][f"{element}-{conf}"] = acwf_oxides_results["BM_fit_data"][f"{element}-{conf}"]
        all_oxides_results["num_atoms_in_sim_cell"][f"{element}-{conf}"] = acwf_oxides_results["num_atoms_in_sim_cell"][f"{element}-{conf}"]

In [15]:
with open("results-oxides-verification-PBE-v1-quantum_espresso-SSSP-1.3.0-PBE-precision-tmp.json", "w") as f:
    json.dump(all_oxides_results, f, indent=4)